<a href="https://colab.research.google.com/github/shubham8garg/MCP_A2A_session/blob/main/MCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai mcp fastmcp httpx python-dotenv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.1/356.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.4 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-abc"
# Verify
assert os.environ.get("OPENAI_API_KEY", "").startswith("sk-"), "❌ Please set a valid OPENAI_API_KEY"
print("✅ OPENAI_API_KEY is set")

✅ OPENAI_API_KEY is set


In [3]:
# Cell 3: Common imports we'll use throughout
import os
import json
import time
import asyncio
from openai import OpenAI

# Initialize OpenAI client (we'll reuse this everywhere)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL = "gpt-5.2"  # Change to "gpt-5-mini" for better results

print(f"✅ OpenAI client initialized")
print(f"📌 Model: {MODEL}")

✅ OpenAI client initialized
📌 Model: gpt-5.2


In [4]:
# Cell 4: Pretty printing helpers (used across all parts)

def print_header(text):
    """Print a prominent section header"""
    width = 60
    print("\n" + "═" * width)
    print(f"  {text}")
    print("═" * width)

def print_step(num, text):
    """Print a numbered step"""
    print(f"\n▶ Step {num}: {text}")
    print(f"{'─' * 55}")

def print_tool_call(name, args):
    """Print a tool call with its arguments"""
    print(f"  📤 TOOL CALL: {name}")
    if isinstance(args, str):
        args = json.loads(args)
    for k, v in args.items():
        print(f"     {k}: {v}")

def print_tool_result(text, max_lines=8):
    """Print a preview of tool results"""
    lines = text.split("\n")
    for line in lines[:max_lines]:
        print(f"  📥 {line}")
    if len(lines) > max_lines:
        print(f"  📥 ... ({len(lines) - max_lines} more lines)")

print("✅ Helper functions defined")

✅ Helper functions defined


# 📖 PART 1: Raw LLM Call — "The Smart but Blind Assistant"

**Goal:** Show that an LLM without tools produces hallucinated, ungrounded responses.

We'll ask it to plan a real trip — and see how it makes up prices, flights, and hotels.

In [5]:
# Cell 6: The user's travel planning request
# We'll reuse this SAME query across all 3 parts to compare results

USER_QUERY = """
Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.
I prefer window seats, vegetarian food, and boutique hotels.
Search real flights and hotels with actual prices.
Book everything if it looks good.
"""

print("💬 User Query:")
print(USER_QUERY)


💬 User Query:

Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.
I prefer window seats, vegetarian food, and boutique hotels.
Search real flights and hotels with actual prices.
Book everything if it looks good.



In [6]:
# Cell 7: Call the LLM with NO tools — just raw text generation

print_header("PART 1: Raw LLM Call (No Tools)")
print("\n🧠 Sending query to LLM with NO tools...")
print("⏳ Waiting for response...\n")

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a travel planning assistant. Answer the user's travel planning request.",
        },
        {
            "role": "user",
            "content": USER_QUERY,
        },
    ],
)

elapsed = time.time() - start_time
raw_answer = response.choices[0].message.content

print(f"⏱️  Response received in {elapsed:.1f}s\n")
print("─" * 55)
print(raw_answer)
print("─" * 55)


════════════════════════════════════════════════════════════
  PART 1: Raw LLM Call (No Tools)
════════════════════════════════════════════════════════════

🧠 Sending query to LLM with NO tools...
⏳ Waiting for response...

⏱️  Response received in 35.8s

───────────────────────────────────────────────────────
I can plan the full 5-day Tokyo itinerary, flight/hotel shortlists, and a booking-ready checklist within your $3,000 budget—but I **can’t search live inventory, see real-time prices, or book** flights/hotels directly from here. What I *can* do is: (1) propose the best-value options based on typical October pricing patterns and reputable providers, (2) give you **exact searches/links and filters** to pull current prices in minutes, and (3) once you paste the top options you see (or screenshots), I’ll tell you **which to book** and how to keep the total under budget.

To “search real flights and hotels with actual prices,” I’ll need either:
- you to run the searches and share resu

In [7]:
# Cell 8: Highlight what's wrong with the raw response

print_header("🔍 ANALYSIS: What's wrong with this response?")

problems = [
    ("Prices are HALLUCINATED",     "The LLM made up flight/hotel prices — no API was called"),
    ("No real availability",         "It doesn't know if those flights/hotels actually exist"),
    ("No calendar awareness",        "It doesn't know your schedule — could suggest busy dates"),
    ("Cannot actually book",         "It's just TEXT — no actions taken, no confirmation numbers"),
    ("No personalization",           "Doesn't know your loyalty programs or past trip history"),
    ("No budget validation",         "Never checked if the plan actually fits within $3000"),
]

for i, (problem, detail) in enumerate(problems, 1):
    print(f"\n  ❌ Problem {i}: {problem}")
    print(f"     → {detail}")

print(f"\n{'═' * 55}")
print("  💡 SOLUTION: Give the LLM REAL TOOLS via MCP!")
print("  👉 Let's see this in Part 2...")
print(f"{'═' * 55}")


════════════════════════════════════════════════════════════
  🔍 ANALYSIS: What's wrong with this response?
════════════════════════════════════════════════════════════

  ❌ Problem 1: Prices are HALLUCINATED
     → The LLM made up flight/hotel prices — no API was called

  ❌ Problem 2: No real availability
     → It doesn't know if those flights/hotels actually exist

  ❌ Problem 3: No calendar awareness
     → It doesn't know your schedule — could suggest busy dates

  ❌ Problem 4: Cannot actually book
     → It's just TEXT — no actions taken, no confirmation numbers

  ❌ Problem 5: No personalization
     → Doesn't know your loyalty programs or past trip history

  ❌ Problem 6: No budget validation
     → Never checked if the plan actually fits within $3000

═══════════════════════════════════════════════════════
  💡 SOLUTION: Give the LLM REAL TOOLS via MCP!
  👉 Let's see this in Part 2...
═══════════════════════════════════════════════════════


# 📖 PART 2: LLM + Kiwi.com MCP Server

**Goal:** Connect to a **real, public MCP server** (Kiwi.com) and let the LLM search for actual flights.

**What is MCP?**
- **Model Context Protocol** — an open standard by Anthropic
- Think of it as **"USB-C for AI"** — universal way to plug tools/data into LLMs
- Client ↔ Server architecture
- Server exposes **Tools** (functions), **Resources** (data), **Prompts** (templates)

**What we'll do:**
1. Connect to Kiwi.com's MCP server
2. Discover what tools they expose
3. Let the LLM decide which tools to call
4. Execute the tools via MCP
5. Feed real results back to the LLM

In [8]:
# Cell 10: Import MCP client libraries

from mcp.client.session import ClientSession
from mcp.client.streamable_http import streamablehttp_client

# Kiwi.com's public MCP server endpoint
KIWI_MCP_URL = "https://mcp.kiwi.com/mcp"

print("✅ MCP client libraries imported")
print(f"🌐 Kiwi.com MCP URL: {KIWI_MCP_URL}")

✅ MCP client libraries imported
🌐 Kiwi.com MCP URL: https://mcp.kiwi.com/mcp


In [10]:
# Cell 11: Function to convert MCP tools → OpenAI function-calling format
#
# MCP tools have their own schema format.
# OpenAI expects tools in a specific format.
# This function bridges the two.

def mcp_tools_to_openai_functions(mcp_tools):
    """Convert MCP tool definitions to OpenAI function calling format."""
    openai_tools = []
    for tool in mcp_tools:
        openai_tool = {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description or "",
                "parameters": tool.inputSchema if tool.inputSchema else {
                    "type": "object",
                    "properties": {},
                },
            },
        }
        openai_tools.append(openai_tool)
    return openai_tools

print("✅ Tool converter function defined")
print()
print("📐 MCP Tool Format        →    OpenAI Tool Format")
print("   tool.name               →    function.name")
print("   tool.description        →    function.description")
print("   tool.inputSchema        →    function.parameters")

✅ Tool converter function defined

📐 MCP Tool Format        →    OpenAI Tool Format
   tool.name               →    function.name
   tool.description        →    function.description
   tool.inputSchema        →    function.parameters


In [11]:
# Cell 12: The agentic loop — LLM decides, we execute, repeat
#
# This is the CORE pattern for tool-use with any LLM:
#   1. Send messages + tools to LLM
#   2. If LLM wants to call a tool → execute it via MCP → add result to messages
#   3. If LLM is done → return final answer
#   4. Repeat

async def run_agentic_loop(session, messages, openai_tools, max_iterations=10):
    """
    Run the LLM ↔ MCP tool-use loop.

    Args:
        session: Active MCP ClientSession
        messages: Conversation messages list
        openai_tools: Tools in OpenAI format
        max_iterations: Safety limit

    Returns:
        final_answer (str), stats (dict)
    """
    stats = {"iterations": 0, "tool_calls": 0, "tools_used": set()}

    for iteration in range(1, max_iterations + 1):
        stats["iterations"] = iteration
        print(f"\n🔄 Iteration {iteration}")
        print(f"{'─' * 45}")

        # Call OpenAI
        iter_start = time.time()
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=openai_tools if openai_tools else None,
        )
        iter_elapsed = time.time() - iter_start

        choice = response.choices[0]
        message = choice.message

        print(f"  ⏱️  LLM responded in {iter_elapsed:.1f}s | Finish: {choice.finish_reason}")

        # ─── CASE A: LLM wants to call tool(s) ───
        if choice.finish_reason == "tool_calls" and message.tool_calls:
            num_calls = len(message.tool_calls)
            stats["tool_calls"] += num_calls
            print(f"  🔧 LLM wants {num_calls} tool call(s)")

            # Add assistant message to conversation
            messages.append(message.model_dump())

            # Execute each tool via MCP
            for tool_call in message.tool_calls:
                func_name = tool_call.function.name
                func_args = json.loads(tool_call.function.arguments)
                stats["tools_used"].add(func_name)

                print()
                print_tool_call(func_name, func_args)

                try:
                    # ⭐ THIS IS THE KEY: Execute tool via MCP protocol
                    mcp_result = await session.call_tool(func_name, func_args)

                    result_text = ""
                    for block in mcp_result.content:
                        if hasattr(block, "text"):
                            result_text += block.text

                    print(f"  📥 Result received ({len(result_text)} chars)")
                    # Show brief preview
                    preview = result_text[:200].replace('\n', ' ')
                    print(f"  📄 Preview: {preview}...")

                except Exception as e:
                    result_text = json.dumps({"error": str(e)})
                    print(f"  ❌ Error: {e}")

                # Add tool result to conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result_text,
                })

        # ─── CASE B: LLM is done ───
        elif choice.finish_reason == "stop":
            print(f"  ✅ LLM finished!")
            return message.content or "No response", stats

        # ─── CASE C: Unexpected ───
        else:
            print(f"  ⚠️  Unexpected: {choice.finish_reason}")
            if message.content:
                return message.content, stats
            messages.append(message.model_dump())

    return "Max iterations reached", stats

print("✅ Agentic loop function defined")

✅ Agentic loop function defined


In [12]:
# Cell 13: Connect to Kiwi.com MCP server and discover tools

print_header("PART 2: Connecting to Kiwi.com MCP Server")

async def discover_kiwi_tools():
    """Connect to Kiwi and list available tools."""
    async with streamablehttp_client(url=KIWI_MCP_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            print("✅ Connected to Kiwi.com MCP server!\n")

            # Discover tools
            tools_response = await session.list_tools()

            print(f"📋 Found {len(tools_response.tools)} tools:\n")
            for i, tool in enumerate(tools_response.tools, 1):
                desc = (tool.description or "No description")[:80]
                print(f"  {i}. 🔧 {tool.name}")
                print(f"     {desc}")
                print()

            return tools_response.tools

kiwi_tools = await discover_kiwi_tools()


════════════════════════════════════════════════════════════
  PART 2: Connecting to Kiwi.com MCP Server
════════════════════════════════════════════════════════════
✅ Connected to Kiwi.com MCP server!

📋 Found 2 tools:

  1. 🔧 search-flight
     
# Search for a flight

## Description

Uses the Kiwi API to search for availabl

  2. 🔧 feedback-to-devs
     Send feedback to the dev of the Kiwi MCP server.



In [13]:
# Cell 14: Now run the SAME user query — but with Kiwi.com tools!

print_header("PART 2: LLM + Kiwi.com Flight Search")

KIWI_QUERY = """
Search for real flights from San Francisco (SFO) to Tokyo
for October 1st 2026 to October 6th 2026.
Show me the cheapest options with actual prices.
"""

print(f"💬 User Query: {KIWI_QUERY.strip()}\n")

async def run_kiwi_demo():
    async with streamablehttp_client(url=KIWI_MCP_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Get tools
            tools_response = await session.list_tools()
            openai_tools = mcp_tools_to_openai_functions(tools_response.tools)
            print(f"🔧 {len(openai_tools)} tools available to LLM\n")

            # Prepare messages
            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a travel assistant with access to real flight search tools. "
                        "Use the available tools to search for REAL flights. "
                        "Never make up prices — always use tools for real data."
                    ),
                },
                {"role": "user", "content": KIWI_QUERY},
            ]

            # Run the agentic loop
            start = time.time()
            answer, stats = await run_agentic_loop(session, messages, openai_tools)
            total_time = time.time() - start

            return answer, stats, total_time

kiwi_answer, kiwi_stats, kiwi_time = await run_kiwi_demo()


════════════════════════════════════════════════════════════
  PART 2: LLM + Kiwi.com Flight Search
════════════════════════════════════════════════════════════
💬 User Query: Search for real flights from San Francisco (SFO) to Tokyo 
for October 1st 2026 to October 6th 2026.
Show me the cheapest options with actual prices.

🔧 2 tools available to LLM


🔄 Iteration 1
─────────────────────────────────────────────
  ⏱️  LLM responded in 3.0s | Finish: tool_calls
  🔧 LLM wants 1 tool call(s)

  📤 TOOL CALL: search-flight
     flyFrom: SFO
     flyTo: Tokyo
     departureDate: 01/10/2026
     returnDate: 06/10/2026
     passengers: {'adults': 1, 'children': 0, 'infants': 0}
     cabinClass: M
     sort: price
     curr: USD
     locale: en
  📥 Result received (18867 chars)
  📄 Preview: [   {     "flyFrom": "SFO",     "flyTo": "NRT",     "cityFrom": "San Francisco",     "cityTo": "Tokyo",     "departure": {       "utc": "2026-10-01T23:45:00.000Z",       "local": "2026-10-01T16:45:00....

🔄 

In [14]:
# Cell 15: Display the grounded response from Kiwi MCP

print_header("✈️ REAL Flight Search Results (via Kiwi.com MCP)")

print(f"""
┌─ Execution Summary ────────────────────────
│  Iterations:      {kiwi_stats['iterations']}
│  Tool calls:      {kiwi_stats['tool_calls']}
│  Tools used:      {', '.join(kiwi_stats['tools_used'])}
│  Total time:      {kiwi_time:.1f}s
└─────────────────────────────────────────────
""")

print(kiwi_answer)


════════════════════════════════════════════════════════════
  ✈️ REAL Flight Search Results (via Kiwi.com MCP)
════════════════════════════════════════════════════════════

┌─ Execution Summary ────────────────────────
│  Iterations:      2
│  Tool calls:      1
│  Tools used:      search-flight
│  Total time:      18.9s
└─────────────────────────────────────────────

Here are **real, cheapest round‑trip Economy (M) flight options** found for **San Francisco (SFO) → Tokyo** departing **Oct 1, 2026** and returning **Oct 6, 2026** (prices in **USD**).

## Cheapest options (sorted by price)

| Outbound (incl. layovers) | Outbound time (local) & duration | Cabin | Return (incl. layovers) | Return time (local) & duration | Cabin | Total price | Booking link |
|---|---:|---|---|---:|---|---:|---|
| **SFO → NRT** | 10/01 16:45 → 10/02 20:00 (**11h 15m**) | Economy | **NRT → SFO** | 10/06 21:30 → 10/06 14:45 (**9h 15m**) | Economy | **$790** | https://on.kiwi.com/rSyy2h |
| **SFO → HKG → NRT

In [15]:
# Cell 16: Compare Part 1 (raw) vs Part 2 (MCP)

print_header("🔍 COMPARISON: Part 1 vs Part 2")

comparison = [
    ("Prices",       "❌ Made-up/hallucinated",    "✅ REAL from Kiwi.com API"),
    ("Flights",      "❌ Generic suggestions",      "✅ Specific flight data"),
    ("Availability", "❌ No real check",            "✅ Actual availability"),
    ("Tool calls",   "❌ Zero",                     f"✅ {kiwi_stats['tool_calls']} MCP tool calls"),
    ("Grounded",     "❌ Pure hallucination",       "✅ Data-backed response"),
]

print(f"\n  {'Aspect':<15} {'Part 1 (Raw LLM)':<28} {'Part 2 (LLM + MCP)'}")
print(f"  {'─'*15} {'─'*28} {'─'*28}")
for aspect, p1, p2 in comparison:
    print(f"  {aspect:<15} {p1:<28} {p2}")

print(f"""
\n💡 KEY INSIGHT:
   The LLM DISCOVERED the tools, DECIDED which to call,
   and produced a GROUNDED response with real data.
   We didn't hardcode any logic!

👉 Next: Let's build our OWN MCP server...
""")


════════════════════════════════════════════════════════════
  🔍 COMPARISON: Part 1 vs Part 2
════════════════════════════════════════════════════════════

  Aspect          Part 1 (Raw LLM)             Part 2 (LLM + MCP)
  ─────────────── ──────────────────────────── ────────────────────────────
  Prices          ❌ Made-up/hallucinated       ✅ REAL from Kiwi.com API
  Flights         ❌ Generic suggestions        ✅ Specific flight data
  Availability    ❌ No real check              ✅ Actual availability
  Tool calls      ❌ Zero                       ✅ 1 MCP tool calls
  Grounded        ❌ Pure hallucination         ✅ Data-backed response


💡 KEY INSIGHT:
   The LLM DISCOVERED the tools, DECIDED which to call,
   and produced a GROUNDED response with real data.
   We didn't hardcode any logic!

👉 Next: Let's build our OWN MCP server...



# 📖 PART 3: Build a Custom MCP Server + Use with LLM

**Goal:** Build our own MCP server with multiple tools, resources, and prompts — then connect it to the LLM.

**What we'll build:**
- 🔧 **5 Tools**: search_flights, search_hotels, check_calendar, estimate_daily_expenses, calculate_trip_budget
- 📄 **2 Resources**: user preferences, Tokyo travel tips
- 📝 **1 Prompt template**: trip planner system prompt

**Architecture:**

In [16]:
# Cell 18: We'll write our MCP server to a file
# The server runs as a SEPARATE PROCESS — spawned via stdio
# Let's first create the directory

import os
os.makedirs("mcp_server", exist_ok=True)
print("✅ Created mcp_server/ directory")

✅ Created mcp_server/ directory


In [17]:
%%writefile mcp_server/travel_server.py
"""
Travel Planning MCP Server — Built with FastMCP 2.x
"""

import json
import sys
from datetime import datetime

try:
    from fastmcp import FastMCP
except ImportError as e:
    print(f"Import error: {e}", file=sys.stderr)
    sys.exit(1)

# ═══════════════════════════════════════════
# Create the MCP Server (FastMCP 2.x API)
# ═══════════════════════════════════════════
mcp = FastMCP("Travel Planning Tools")

# ═══════════════════════════════════════════
# SIMULATED DATA
# ═══════════════════════════════════════════

FLIGHT_DATABASE = {
    ("SFO", "TYO"): [
        {"airline": "ANA", "flight": "NH7", "depart": "11:30 AM", "arrive": "3:30 PM +1",
         "duration": "11h 00m", "price": 820, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL1", "depart": "1:00 PM", "arrive": "5:15 PM +1",
         "duration": "11h 15m", "price": 910, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA837", "depart": "2:45 PM", "arrive": "7:00 PM +1",
         "duration": "11h 15m", "price": 750, "stops": 0, "window_seat": False},
        {"airline": "Delta (via Seoul)", "flight": "DL295", "depart": "9:00 AM", "arrive": "8:30 PM +1",
         "duration": "17h 30m", "price": 620, "stops": 1, "window_seat": True},
    ],
    ("TYO", "SFO"): [
        {"airline": "ANA", "flight": "NH8", "depart": "5:00 PM", "arrive": "10:30 AM same day",
         "duration": "9h 30m", "price": 840, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL2", "depart": "6:30 PM", "arrive": "11:45 AM same day",
         "duration": "9h 15m", "price": 890, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA838", "depart": "4:00 PM", "arrive": "9:30 AM same day",
         "duration": "9h 30m", "price": 770, "stops": 0, "window_seat": True},
    ],
}

HOTEL_DATABASE = {
    "Tokyo": [
        {"name": "K5 Tokyo", "area": "Nihonbashi", "type": "boutique",
         "rating": 4.8, "price_night": 190, "vegetarian": True},
        {"name": "Trunk Hotel", "area": "Shibuya", "type": "boutique",
         "rating": 4.6, "price_night": 220, "vegetarian": True},
        {"name": "MUJI Hotel Ginza", "area": "Ginza", "type": "boutique",
         "rating": 4.5, "price_night": 175, "vegetarian": True},
        {"name": "APA Hotel Shinjuku", "area": "Shinjuku", "type": "business",
         "rating": 3.8, "price_night": 95, "vegetarian": False},
        {"name": "Park Hyatt Tokyo", "area": "Shinjuku", "type": "luxury",
         "rating": 4.9, "price_night": 550, "vegetarian": True},
    ]
}

CALENDAR = [
    {"date": "2026-10-01", "time": "09:00-10:00", "event": "Team standup", "flexible": True},
    {"date": "2026-10-03", "time": "14:00-15:00", "event": "Dentist appointment", "flexible": False},
    {"date": "2026-10-07", "time": "11:00-12:00", "event": "Project review", "flexible": False},
]

USER_PREFS = {
    "name": "Alex Johnson", "home_airport": "SFO",
    "seat": "window", "meal": "vegetarian", "hotel_type": "boutique",
    "budget_style": "moderate",
    "loyalty": ["ANA Mileage Club", "Marriott Bonvoy"],
    "interests": ["Japanese cuisine", "temples", "street photography", "local markets"],
    "past_trips": ["Kyoto 2023", "Osaka 2024"],
}

TOKYO_TIPS = {
    "best_months": ["March-April (Cherry Blossom)", "October-November (Autumn)"],
    "must_visit": ["Senso-ji Temple", "Meiji Shrine", "Tsukiji Market", "Shibuya Crossing"],
    "vegetarian_spots": ["Ain Soph Ripple (Shinjuku)", "T's TanTan (Tokyo Station)", "Bon (Taito)"],
    "tip": "Get a 72-hour Tokyo Metro Pass for 1,500 yen (~$10)",
}


# ═══════════════════════════════════════════
# TOOLS
# ═══════════════════════════════════════════

@mcp.tool()
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search flights between two airports on a specific date. Returns real pricing and availability."""
    origin = origin.upper().strip()
    destination = destination.upper().strip()
    code_map = {"TOKYO": "TYO", "NRT": "TYO", "HND": "TYO", "SAN FRANCISCO": "SFO"}
    origin = code_map.get(origin, origin)
    destination = code_map.get(destination, destination)

    flights = FLIGHT_DATABASE.get((origin, destination), [])
    results = [dict(f, date=date) for f in flights]

    return json.dumps({
        "route": f"{origin} -> {destination}", "date": date,
        "count": len(results), "flights": results, "currency": "USD"
    }, indent=2)


@mcp.tool()
def search_hotels(city: str, checkin: str, checkout: str, hotel_type: str = "any") -> str:
    """Search hotels in a city with pricing. Filter by type: boutique, business, luxury, or any."""
    hotels = HOTEL_DATABASE.get(city.strip().title(), [])

    if hotel_type != "any":
        hotels = [h for h in hotels if h["type"] == hotel_type.lower()]

    try:
        nights = (datetime.strptime(checkout, "%Y-%m-%d") - datetime.strptime(checkin, "%Y-%m-%d")).days
    except ValueError:
        nights = 5

    results = [dict(h, checkin=checkin, checkout=checkout, nights=nights,
                    total_price=h["price_night"] * nights) for h in hotels]
    results.sort(key=lambda x: x["rating"], reverse=True)

    return json.dumps({
        "city": city.title(), "checkin": checkin, "checkout": checkout,
        "nights": nights, "count": len(results), "hotels": results, "currency": "USD"
    }, indent=2)


@mcp.tool()
def check_calendar(start_date: str, end_date: str) -> str:
    """Check user calendar for events and conflicts in a date range."""
    try:
        d1 = datetime.strptime(start_date, "%Y-%m-%d")
        d2 = datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return json.dumps({"error": "Invalid date format. Use YYYY-MM-DD"})

    events_in_range = []
    conflicts = []
    for ev in CALENDAR:
        ev_date = datetime.strptime(ev["date"], "%Y-%m-%d")
        if d1 <= ev_date <= d2:
            events_in_range.append(ev)
            if not ev["flexible"]:
                conflicts.append(ev)

    return json.dumps({
        "range": f"{start_date} to {end_date}",
        "events": events_in_range, "conflicts": conflicts,
        "conflict_count": len(conflicts),
    }, indent=2)


@mcp.tool()
def estimate_daily_expenses(city: str, style: str = "moderate") -> str:
    """Estimate daily expenses for a city. Style: budget, moderate, or luxury."""
    costs = {
        "Tokyo": {
            "budget": {"food": 30, "transport": 10, "activities": 20, "misc": 10},
            "moderate": {"food": 60, "transport": 15, "activities": 40, "misc": 20},
            "luxury": {"food": 150, "transport": 50, "activities": 100, "misc": 50},
        }
    }
    city_costs = costs.get(city.title(), {}).get(style.lower())
    if not city_costs:
        return json.dumps({"error": f"No data for {city}/{style}"})

    city_costs["total_per_day"] = sum(city_costs.values())
    return json.dumps({"city": city.title(), "style": style, "daily_usd": city_costs}, indent=2)


@mcp.tool()
def calculate_trip_budget(
    flight_cost: float, hotel_cost: float, daily_expense: float,
    num_days: int, total_budget: float
) -> str:
    """Calculate total trip cost and check if it fits within budget."""
    daily_total = daily_expense * num_days
    total = flight_cost + hotel_cost + daily_total
    remaining = total_budget - total

    return json.dumps({
        "breakdown": {"flights": flight_cost, "hotel": hotel_cost,
                      "daily_expenses": daily_total, "total": total},
        "budget": total_budget, "remaining": remaining,
        "feasible": remaining >= 0,
        "verdict": (f"Within budget! ${remaining:.0f} left." if remaining >= 0
                    else f"Over by ${abs(remaining):.0f}.")
    }, indent=2)


# ═══════════════════════════════════════════
# RESOURCES
# ═══════════════════════════════════════════

@mcp.resource("user://travel-preferences")
def get_user_preferences() -> str:
    """User travel preferences and profile."""
    return json.dumps(USER_PREFS, indent=2)


@mcp.resource("data://tokyo-travel-tips")
def get_tokyo_tips() -> str:
    """Local travel tips for Tokyo."""
    return json.dumps(TOKYO_TIPS, indent=2)


# ═══════════════════════════════════════════
# PROMPTS
# ═══════════════════════════════════════════

@mcp.prompt()
def trip_planner_prompt(destination: str, days: int, budget: float) -> str:
    """System prompt for comprehensive trip planning."""
    return f"""You are an expert travel planner. Plan a {days}-day trip to {destination} within ${budget}.

Steps:
1. Read user://travel-preferences for personalization
2. Use check_calendar to find scheduling conflicts
3. Use search_flights for real flight options
4. Use search_hotels matching user hotel type preference
5. Use estimate_daily_expenses for cost estimation
6. Use calculate_trip_budget to verify feasibility
7. Compile a comprehensive, personalized plan

IMPORTANT: Use REAL data from tools. Never make up prices."""


# ═══════════════════════════════════════════
# Run
# ═══════════════════════════════════════════
if __name__ == "__main__":
    mcp.run(transport="stdio")

Writing mcp_server/travel_server.py


In [19]:
# Cell 20: Verify the server file is valid and runs without errors

import subprocess, sys, os

server_path = os.path.abspath("mcp_server/travel_server.py")

print(f"📄 Server: {server_path}")
print(f"📂 Exists: {os.path.exists(server_path)}\n")

# Test: can it start without crashing?
result = subprocess.run(
    [sys.executable, server_path],
    capture_output=True, text=True, timeout=5, input=""
)

if result.returncode == 0 or "Traceback" not in result.stderr:
    print("✅ Server starts cleanly! No import/syntax errors.")

    # Count what we defined
    with open(server_path) as f:
        content = f.read()
    tools = content.count("@mcp.tool()")
    resources = content.count("@mcp.resource(")
    prompts = content.count("@mcp.prompt()")
    print(f"\n📦 Server contents:")
    print(f"   🔧 Tools:     {tools}")
    print(f"   📄 Resources: {resources}")
    print(f"   📝 Prompts:   {prompts}")
else:
    print(f"❌ Server has errors:\n")
    print(result.stderr)

📄 Server: /content/mcp_server/travel_server.py
📂 Exists: True

✅ Server starts cleanly! No import/syntax errors.

📦 Server contents:
   🔧 Tools:     5
   📄 Resources: 2
   📝 Prompts:   1


In [20]:
# Cell 21: Import MCP stdio client (to spawn our server as a subprocess)

import sys
from mcp.client.stdio import stdio_client, StdioServerParameters

# Configure how to launch our server
SERVER_PARAMS = StdioServerParameters(
    command=sys.executable,               # Same Python interpreter
    args=["mcp_server/travel_server.py"], # Our server script
    env=None,                             # Inherit environment
)

print("✅ Stdio client imported")
print(f"📌 Python: {sys.executable}")
print(f"📌 Server: mcp_server/travel_server.py")
print(f"\n💡 The client will SPAWN the server as a subprocess via stdio")
print(f"   No need to start the server separately!")

✅ Stdio client imported
📌 Python: /usr/bin/python3
📌 Server: mcp_server/travel_server.py

💡 The client will SPAWN the server as a subprocess via stdio
   No need to start the server separately!


In [25]:
# Cell 22: Connect to our MCP server and discover tools

print_header("Connecting to Our Custom MCP Server")

async def discover_custom_tools():
    import sys
    import os
    # We will pass this directly to stdio_client's errlog argument
    devnull_stderr_file = None
    try:
        if os.name == 'posix':
            devnull_stderr_file = open(os.devnull, 'w')

        # Pass the /dev/null file object directly to stdio_client's errlog
        async with stdio_client(SERVER_PARAMS, errlog=devnull_stderr_file) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                print("✅ Server spawned and connected!\n")

                # ── Discover Tools ──
                tools_result = await session.list_tools()
                print(f"🔧 TOOLS ({len(tools_result.tools)}):")
                for t in tools_result.tools:
                    print(f"   • {t.name}: {(t.description or '')[:60]}")

                # ── Discover Resources ──
                resources_result = await session.list_resources()
                print(f"\n📄 RESOURCES ({len(resources_result.resources)}):")
                for r in resources_result.resources:
                    print(f"   • {r.uri}")

                # ── Discover Prompts ──
                prompts_result = await session.list_prompts()
                print(f"\n📝 PROMPTS ({len(prompts_result.prompts)}):")
                for p in prompts_result.prompts:
                    print(f"   • {p.name}: {(p.description or '')[:60]}")

                return tools_result.tools
    finally:
        if devnull_stderr_file:
            devnull_stderr_file.close()

custom_tools = await discover_custom_tools()



════════════════════════════════════════════════════════════
  Connecting to Our Custom MCP Server
════════════════════════════════════════════════════════════
✅ Server spawned and connected!

🔧 TOOLS (5):
   • search_flights: Search flights between two airports on a specific date. Retu
   • search_hotels: Search hotels in a city with pricing. Filter by type: boutiq
   • check_calendar: Check user calendar for events and conflicts in a date range
   • estimate_daily_expenses: Estimate daily expenses for a city. Style: budget, moderate,
   • calculate_trip_budget: Calculate total trip cost and check if it fits within budget

📄 RESOURCES (2):
   • user://travel-preferences
   • data://tokyo-travel-tips

📝 PROMPTS (1):
   • trip_planner_prompt: System prompt for comprehensive trip planning.


In [27]:
# Cell 23: Read resources from our MCP server

print_header("Reading MCP Resources")

async def read_resources():
    import os
    devnull_stderr_file = None
    try:
        if os.name == 'posix':
            devnull_stderr_file = open(os.devnull, 'w')

        async with stdio_client(SERVER_PARAMS, errlog=devnull_stderr_file) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()

                # ── Read User Preferences ──
                print("📄 Resource: user://travel-preferences\n")
                prefs = await session.read_resource("user://travel-preferences")
                prefs_text = prefs.contents[0].text if prefs.contents else "{}"
                prefs_data = json.loads(prefs_text)

                print(f"   👤 Name:     {prefs_data.get('name')}")
                print(f"   🏠 Airport:  {prefs_data.get('home_airport')}")
                print(f"   💺 Seat:     {prefs_data.get('seat')}")
                print(f"   🍱 Meal:     {prefs_data.get('meal')}")
                print(f"   🏨 Hotel:    {prefs_data.get('hotel_type')}")
                print(f"   🎯 Interests: {', '.join(prefs_data.get('interests', []))}")

                # ── Read Tokyo Tips ──
                print(f"\n📄 Resource: data://tokyo-travel-tips\n")
                tips = await session.read_resource("data://tokyo-travel-tips")
                tips_text = tips.contents[0].text if tips.contents else "{}"
                tips_data = json.loads(tips_text)

                print(f"   🗓️  Best months: {', '.join(tips_data.get('best_months', []))}")
                print(f"   📍 Must visit:  {', '.join(tips_data.get('must_visit', []))}")
                print(f"   🍱 Veg spots:   {', '.join(tips_data.get('vegetarian_spots', []))}")
                print(f"   💡 Tip:         {tips_data.get('tip')}")

                return prefs_text, tips_text
    finally:
        if devnull_stderr_file:
            devnull_stderr_file.close()

user_prefs_text, tokyo_tips_text = await read_resources()



════════════════════════════════════════════════════════════
  Reading MCP Resources
════════════════════════════════════════════════════════════
📄 Resource: user://travel-preferences

   👤 Name:     Alex Johnson
   🏠 Airport:  SFO
   💺 Seat:     window
   🍱 Meal:     vegetarian
   🏨 Hotel:    boutique
   🎯 Interests: Japanese cuisine, temples, street photography, local markets

📄 Resource: data://tokyo-travel-tips

   🗓️  Best months: March-April (Cherry Blossom), October-November (Autumn)
   📍 Must visit:  Senso-ji Temple, Meiji Shrine, Tsukiji Market, Shibuya Crossing
   🍱 Veg spots:   Ain Soph Ripple (Shinjuku), T's TanTan (Tokyo Station), Bon (Taito)
   💡 Tip:         Get a 72-hour Tokyo Metro Pass for 1,500 yen (~$10)


In [29]:
# Cell 24: Get the prompt template from MCP server

print_header("Getting MCP Prompt Template")

async def get_prompt_template():
    import os
    devnull_stderr_file = None
    try:
        if os.name == 'posix':
            devnull_stderr_file = open(os.devnull, 'w')

        async with stdio_client(SERVER_PARAMS, errlog=devnull_stderr_file) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()

                result = await session.get_prompt(
                    "trip_planner_prompt",
                    arguments={"destination": "Tokyo", "days": "5", "budget": "3000"},
                )

                prompt_text = ""
                for msg in result.messages:
                    if hasattr(msg.content, "text"):
                        prompt_text = msg.content.text

                print("📝 Generated System Prompt:\n")
                print(f"   {prompt_text}")
                return prompt_text
    finally:
        if devnull_stderr_file:
            devnull_stderr_file.close()

system_prompt = await get_prompt_template()



════════════════════════════════════════════════════════════
  Getting MCP Prompt Template
════════════════════════════════════════════════════════════
📝 Generated System Prompt:

   You are an expert travel planner. Plan a 5-day trip to Tokyo within $3000.0.

Steps:
1. Read user://travel-preferences for personalization
2. Use check_calendar to find scheduling conflicts
3. Use search_flights for real flight options
4. Use search_hotels matching user hotel type preference
5. Use estimate_daily_expenses for cost estimation
6. Use calculate_trip_budget to verify feasibility
7. Compile a comprehensive, personalized plan

IMPORTANT: Use REAL data from tools. Never make up prices.


In [31]:
# Cell 25: Test individual MCP tool calls with 2026 dates

print_header("Testing Individual MCP Tool Calls")

async def test_tools():
    import os
    devnull_stderr_file = None
    try:
        if os.name == 'posix':
            devnull_stderr_file = open(os.devnull, 'w')

        async with stdio_client(SERVER_PARAMS, errlog=devnull_stderr_file) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()

                # Test 1: Search flights
                print("🔧 Test: search_flights('SFO', 'TYO', '2026-10-01')")
                result = await session.call_tool("search_flights", {
                    "origin": "SFO", "destination": "TYO", "date": "2026-10-01"
                })
                flights = json.loads(result.content[0].text)
                print(f"   ✅ Found {flights['count']} flights")
                for f in flights['flights'][:2]:
                    print(f"      ✈️  {f['airline']} {f['flight']} — ${f['price']}")

                # Test 2: Check calendar
                print(f"\n🔧 Test: check_calendar('2026-10-01', '2026-10-06')")
                result = await session.call_tool("check_calendar", {
                    "start_date": "2026-10-01", "end_date": "2026-10-06"
                })
                cal = json.loads(result.content[0].text)
                print(f"   ✅ Found {len(cal['events'])} events, {cal['conflict_count']} conflicts")
                for c in cal['conflicts']:
                    print(f"      ⚠️  {c['date']} {c['time']}: {c['event']}")

                # Test 3: Search hotels
                print(f"\n🔧 Test: search_hotels('Tokyo', ..., hotel_type='boutique')")
                result = await session.call_tool("search_hotels", {
                    "city": "Tokyo", "checkin": "2026-10-01",
                    "checkout": "2026-10-06", "hotel_type": "boutique"
                })
                hotels = json.loads(result.content[0].text)
                print(f"   ✅ Found {hotels['count']} boutique hotels ({hotels['nights']} nights)")
                for h in hotels['hotels'][:2]:
                    print(f"      🏨 {h['name']} — ${h['price_night']}/night (${h['total_price']} total)")

                # Test 4: Estimate expenses
                print(f"\n🔧 Test: estimate_daily_expenses('Tokyo', 'moderate')")
                result = await session.call_tool("estimate_daily_expenses", {
                    "city": "Tokyo", "style": "moderate"
                })
                expenses = json.loads(result.content[0].text)
                daily = expenses['daily_usd']
                print(f"   ✅ Daily estimate: ${daily['total_per_day']}/day")
                print(f"      Food: ${daily['food']} | Transport: ${daily['transport']} | Activities: ${daily['activities']}")

                # Test 5: Budget calculator
                print(f"\n🔧 Test: calculate_trip_budget(...)")
                result = await session.call_tool("calculate_trip_budget", {
                    "flight_cost": 820, "hotel_cost": 950,
                    "daily_expense": 135, "num_days": 5, "total_budget": 3000
                })
                budget = json.loads(result.content[0].text)
                print(f"   ✅ {budget['verdict']}")
                print(f"      Total: ${budget['breakdown']['total']} | Budget: ${budget['budget']} | Remaining: ${budget['remaining']}")
    finally:
        if devnull_stderr_file:
            devnull_stderr_file.close()

await test_tools()
print("\n✅ All 5 tools working! Now let's let the LLM use them...")



════════════════════════════════════════════════════════════
  Testing Individual MCP Tool Calls
════════════════════════════════════════════════════════════
🔧 Test: search_flights('SFO', 'TYO', '2026-10-01')
   ✅ Found 4 flights
      ✈️  ANA NH7 — $820
      ✈️  JAL JL1 — $910

🔧 Test: check_calendar('2026-10-01', '2026-10-06')
   ✅ Found 2 events, 1 conflicts
      ⚠️  2026-10-03 14:00-15:00: Dentist appointment

🔧 Test: search_hotels('Tokyo', ..., hotel_type='boutique')
   ✅ Found 3 boutique hotels (5 nights)
      🏨 K5 Tokyo — $190/night ($950 total)
      🏨 Trunk Hotel — $220/night ($1100 total)

🔧 Test: estimate_daily_expenses('Tokyo', 'moderate')
   ✅ Daily estimate: $135/day
      Food: $60 | Transport: $15 | Activities: $40

🔧 Test: calculate_trip_budget(...)
   ✅ Within budget! $555 left.
      Total: $2445.0 | Budget: $3000.0 | Remaining: $555.0

✅ All 5 tools working! Now let's let the LLM use them...


## 🧠 The Main Event: LLM + Our Custom MCP Server

Now we let the LLM:
1. **Read** our resources (user preferences, Tokyo tips)
2. **Decide** which tools to call and in what order
3. **Execute** tools via MCP
4. **Combine** all results into a comprehensive travel plan

We write **ZERO orchestration logic** — the LLM figures out the workflow!

In [33]:
# Cell 27: The FULL demo — LLM + Our Custom MCP Server

print_header("PART 3: Full Demo — LLM + Custom MCP Server")

print(f"💬 User Query: {USER_QUERY.strip()}\n")

async def run_full_custom_demo():
    import os
    devnull_stderr_file = None
    try:
        if os.name == 'posix':
            devnull_stderr_file = open(os.devnull, 'w')

        async with stdio_client(SERVER_PARAMS, errlog=devnull_stderr_file) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                print("✅ MCP server connected\n")

                # ── Get tools ──
                tools_result = await session.list_tools()
                openai_tools = mcp_tools_to_openai_functions(tools_result.tools)
                print(f"🔧 {len(openai_tools)} tools loaded")

                # ── Read resources for context ──
                prefs = await session.read_resource("user://travel-preferences")
                prefs_text = prefs.contents[0].text

                tips = await session.read_resource("data://tokyo-travel-tips")
                tips_text = tips.contents[0].text
                print(f"📄 2 resources loaded")

                # ── Get prompt template ──
                prompt_result = await session.get_prompt(
                    "trip_planner_prompt",
                    arguments={"destination": "Tokyo", "days": "5", "budget": "3000"},
                )
                system_prompt = prompt_result.messages[0].content.text
                print(f"📝 Prompt template loaded")

                # ── Build system message with all context ──
                full_system = f"""{system_prompt}

## User Preferences (from MCP resource):
{prefs_text}

## Tokyo Travel Tips (from MCP resource):
{tips_text}
"""

                messages = [
                    {"role": "system", "content": full_system},
                    {"role": "user", "content": USER_QUERY},
                ]

                print(f"\n📝 System message: {len(full_system)} chars")
                print(f"🧠 Model: {MODEL}")
                print(f"\n{'═' * 55}")
                print(f"  Starting Agentic Loop...")
                print(f"{'═' * 55}")

                # ── Run the agentic loop ──
                start = time.time()
                answer, stats = await run_agentic_loop(session, messages, openai_tools)
                total_time = time.time() - start

                return answer, stats, total_time
    finally:
        if devnull_stderr_file:
            devnull_stderr_file.close()

final_answer, final_stats, final_time = await run_full_custom_demo()



════════════════════════════════════════════════════════════
  PART 3: Full Demo — LLM + Custom MCP Server
════════════════════════════════════════════════════════════
💬 User Query: Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.
I prefer window seats, vegetarian food, and boutique hotels.
Search real flights and hotels with actual prices.
Book everything if it looks good.

✅ MCP server connected

🔧 5 tools loaded
📄 2 resources loaded
📝 Prompt template loaded

📝 System message: 1358 chars
🧠 Model: gpt-5.2

═══════════════════════════════════════════════════════
  Starting Agentic Loop...
═══════════════════════════════════════════════════════

🔄 Iteration 1
─────────────────────────────────────────────
  ⏱️  LLM responded in 2.2s | Finish: tool_calls
  🔧 LLM wants 2 tool call(s)

  📤 TOOL CALL: check_calendar
     start_date: 2026-10-01
     end_date: 2026-10-31
  📥 Result received (745 chars)
  📄 Preview: {   "range": "2026-10-01 to 2

In [34]:
# Cell 28: Display the complete, grounded travel plan

print_header("📋 COMPLETE TRAVEL PLAN")

print(f"""
┌─ Execution Summary ─────────────────────────────
│  🔄 Iterations:       {final_stats['iterations']}
│  🔧 Total tool calls: {final_stats['tool_calls']}
│  🛠️  Tools used:       {', '.join(final_stats['tools_used'])}
│  ⏱️  Total time:       {final_time:.1f}s
│  📄 Resources read:   2 (preferences + tips)
│  📝 Prompts used:     1 (trip_planner_prompt)
└──────────────────────────────────────────────────
""")

print(final_answer)


════════════════════════════════════════════════════════════
  📋 COMPLETE TRAVEL PLAN
════════════════════════════════════════════════════════════

┌─ Execution Summary ─────────────────────────────
│  🔄 Iterations:       5
│  🔧 Total tool calls: 7
│  🛠️  Tools used:       estimate_daily_expenses, check_calendar, calculate_trip_budget, search_flights, search_hotels
│  ⏱️  Total time:       38.6s
│  📄 Resources read:   2 (preferences + tips)
│  📝 Prompts used:     1 (trip_planner_prompt)
└──────────────────────────────────────────────────

## Tokyo (5 days) — October 2026 — Budget: $3000 (from SFO)
Personalized for **Alex Johnson**: window seat preference, **vegetarian meals**, **boutique hotels**, moderate spending; interests: **Japanese cuisine, temples, street photography, local markets**. You’ve also been to Kyoto/Osaka recently, so this focuses on Tokyo neighborhoods and day-to-day texture.

---

## 1) Date pick + calendar conflicts
I checked your October 2026 calendar. Two **non-

In [35]:
# Cell 29: The Grand Comparison — All 3 Parts

print_header("🏆 GRAND COMPARISON: Part 1 → Part 2 → Part 3")

data = [
    ("Real prices",       "❌ Hallucinated",   "✅ Kiwi.com API",  "✅ Our MCP tools"),
    ("Tools used",        "0",                 f"{kiwi_stats['tool_calls']}",  f"{final_stats['tool_calls']}"),
    ("User preferences",  "❌ None",           "❌ None",          "✅ MCP Resource"),
    ("Calendar check",    "❌ None",           "❌ None",          "✅ MCP Tool"),
    ("Budget validation", "❌ None",           "❌ None",          "✅ MCP Tool"),
    ("Hotel search",      "❌ Generic",        "❌ N/A",           "✅ MCP Tool"),
    ("Personalized",      "❌ No",             "❌ No",            "✅ Yes (prefs + tips)"),
    ("Prompt template",   "❌ Hardcoded",      "❌ Hardcoded",     "✅ MCP Prompt"),
]

print(f"\n  {'Feature':<20} {'Part 1: Raw LLM':<18} {'Part 2: Kiwi MCP':<18} {'Part 3: Custom MCP'}")
print(f"  {'─'*20} {'─'*18} {'─'*18} {'─'*18}")
for row in data:
    print(f"  {row[0]:<20} {row[1]:<18} {row[2]:<18} {row[3]}")

print()


════════════════════════════════════════════════════════════
  🏆 GRAND COMPARISON: Part 1 → Part 2 → Part 3
════════════════════════════════════════════════════════════

  Feature              Part 1: Raw LLM    Part 2: Kiwi MCP   Part 3: Custom MCP
  ──────────────────── ────────────────── ────────────────── ──────────────────
  Real prices          ❌ Hallucinated     ✅ Kiwi.com API     ✅ Our MCP tools
  Tools used           0                  1                  7
  User preferences     ❌ None             ❌ None             ✅ MCP Resource
  Calendar check       ❌ None             ❌ None             ✅ MCP Tool
  Budget validation    ❌ None             ❌ None             ✅ MCP Tool
  Hotel search         ❌ Generic          ❌ N/A              ✅ MCP Tool
  Personalized         ❌ No               ❌ No               ✅ Yes (prefs + tips)
  Prompt template      ❌ Hardcoded        ❌ Hardcoded        ✅ MCP Prompt



In [36]:
# Cell 30: Architecture & Key Takeaways

print_header("🏗️ Architecture: What We Built")

print("""
    ┌──────────────────┐
    │      USER        │
    │  "Plan my trip"  │
    └────────┬─────────┘
             │
             ▼
    ┌───────────────────────────────────────┐
    │        🧠 LLM (GPT-4o-mini)          │
    │                                       │
    │   Reads:   📄 Resources (prefs/tips)  │
    │   Uses:    📝 Prompt template         │
    │   Calls:   🔧 Tools via MCP           │
    └───────┬──────────┬───────────┬────────┘
            │          │           │
         MCP│       MCP│        MCP│
            ▼          ▼           ▼
    ┌───────────┐ ┌─────────┐ ┌──────────┐
    │ search_   │ │ check_  │ │calculate_│
    │ flights   │ │calendar │ │trip_     │
    │ search_   │ │         │ │budget    │
    │ hotels    │ │         │ │estimate_ │
    └───────────┘ └─────────┘ │expenses  │
                               └──────────┘
""")

print("═" * 55)
print("  🔑 KEY TAKEAWAYS")
print("═" * 55)

takeaways = [
    ("MCP = 'USB-C for AI'",
     "Standard protocol to plug tools/data into any LLM"),
    ("3 MCP primitives",
     "Tools (actions), Resources (data), Prompts (templates)"),
    ("LLM decides the workflow",
     "We wrote ZERO orchestration logic — LLM figured it out"),
    ("Server is independent",
     "Runs as separate process, any language, any framework"),
    ("Composable",
     "Mix our server + Kiwi.com + any other MCP server"),
]

for title, desc in takeaways:
    print(f"\n  💡 {title}")
    print(f"     {desc}")

print(f"""

{'═' * 55}
  🔮 WHAT'S NEXT?
{'═' * 55}

  One agent is powerful. But what about MULTIPLE agents
  working together?

  → Finance Agent checking budgets
  → Booking Agent making reservations
  → Local Guide Agent suggesting activities

  That's where A2A (Agent-to-Agent) protocol comes in!

  MCP = Agent ↔ Tools  (vertical integration)
  A2A = Agent ↔ Agent  (horizontal collaboration)
{'═' * 55}
""")


════════════════════════════════════════════════════════════
  🏗️ Architecture: What We Built
════════════════════════════════════════════════════════════

    ┌──────────────────┐
    │      USER        │
    │  "Plan my trip"  │
    └────────┬─────────┘
             │
             ▼
    ┌───────────────────────────────────────┐
    │        🧠 LLM (GPT-4o-mini)          │
    │                                       │
    │   Reads:   📄 Resources (prefs/tips)  │
    │   Uses:    📝 Prompt template         │
    │   Calls:   🔧 Tools via MCP           │
    └───────┬──────────┬───────────┬────────┘
            │          │           │
         MCP│       MCP│        MCP│
            ▼          ▼           ▼
    ┌───────────┐ ┌─────────┐ ┌──────────┐
    │ search_   │ │ check_  │ │calculate_│
    │ flights   │ │calendar │ │trip_     │
    │ search_   │ │         │ │budget    │
    │ hotels    │ │         │ │estimate_ │
    └───────────┘ └─────────┘ │expenses  │
                           

# 🎉 Demo Complete!

## What we demonstrated:

| Part | What | Result |
|------|------|--------|
| **Part 1** | Raw LLM call | ❌ Hallucinated, useless |
| **Part 2** | LLM + Kiwi.com MCP | ✅ Real flight data |
| **Part 3** | LLM + Custom MCP (5 tools, 2 resources, 1 prompt) | ✅ Complete grounded plan |

## MCP gave us:
- 🔧 **Tools** — Real capabilities (search, check, calculate)
- 📄 **Resources** — Context (user prefs, travel tips)
- 📝 **Prompts** — Structure (step-by-step planning)

## The LLM:
- **Discovered** tools automatically
- **Decided** which to call and when
- **Combined** all results into a coherent plan
- We wrote **zero orchestration logic**!